# Lecture 6.2: More on Relational Data and SQL
<div style="border: 1px double black; padding: 10px; margin: 10px">

**After today's lecture you will:**
* Understand how to [filter data using joins](#Filtering-joins)
* Know to execute basic [SQL commands](#SQL-Commands)
</div>

This lecture corresponds to Chapter 13 of your textbook.

In [1]:
library(tidyverse)
library(nycflights13)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.2
✔ tidyr   1.1.1     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



## Filtering joins

Filtering joins allow us to filter rows on one table based on their presence or absence in another table. 

In [2]:
top5 <- count(flights, carrier) %>% top_n(5) %>% print
top5$carrier
flights_top5 = filter(flights, carrier %in% top5$carrier) %>% print

Selecting by n


# A tibble: 5 x 2
  carrier     n
  <chr>   <int>
1 AA      32729
2 B6      54635
3 DL      48110
4 EV      54173
5 UA      58665


[1] "AA" "B6" "DL" "EV" "UA"

# A tibble: 248,312 x 19
    year month   day dep_time sched_dep_time dep_delay arr_time sched_arr_time
   <int> <int> <int>    <int>          <int>     <dbl>    <int>          <int>
 1  2013     1     1      517            515         2      830            819
 2  2013     1     1      533            529         4      850            830
 3  2013     1     1      542            540         2      923            850
 4  2013     1     1      544            545        -1     1004           1022
 5  2013     1     1      554            600        -6      812            837
 6  2013     1     1      554            558        -4      740            728
 7  2013     1     1      555            600        -5      913            854
 8  2013     1     1      557            600        -3      709            723
 9  2013     1     1      557            600        -3      838            846
10  2013     1     1      558            600        -2      753            745
# … with 248,302 more rows,

### Question: 
What if we wanted to filter based on more than one variable? For example, based on busiest days of the year:

One option would be to construct a new string variable which we could then match using the same technique:

But this is cumbersome and will also be slow if you are matching to a large list of choices.

### Semi-join
A better option is to use the built-in command `semi_join`. `semi_join(x, y)` keeps all the observations in `x` that are also in `y`.
![semi join](http://r4ds.had.co.nz/diagrams/join-semi.png)

`semi_join()` takes the same `key=` options as the other join commands.

**Question:** Now let us try to create a table with all flights in the 7 busiest days of the year.

### Exercise
Of the 336,776 flights in `flights`, what % of them departed on one of the 20 busiest days of the year?

### Anti-join
The opposite of a semi-join is an anti-join:
![anti-join](http://r4ds.had.co.nz/diagrams/join-anti.png)

### Exercise
<table><tr><th>Beginner</th><th>Advanced</th></tr>
    <tr>
        <td>How many flights were on planes which are <i>not</i> found in the <code>planes</code> table?</td>
        <td>How many planes flew to destinations which are <i>not</i> found in the <code>airports</code> table?</td>
    </tr>
    </table>

Anti-joins are useful for finding mismatches in joins. Last lecture we saw that some 50k entries in `flights` have `tailnum`s that do not exist in the `planes` table. To find out more about these flights we could use an anti-join to select only those rows:

Do we see any pattern in this table that could explain the missing tail numbers?

Interestingly, the top 5 most frequent `tailnum`s in the overall data are almost the same:

The one plane that is represented is N711M. You can [look it up](https://flightaware.com/resources/registration/N711MQ) in an online database and find out it's a private jet. My guess is the others are private jets too.

# SQL Queries

SQL stands for "Structured Query Language". Many large databases are stored in SQL format, and you will probably encounter one if you work on big data and/or at a large company. 

To introduce SQL we're going to use the `sqldf` package, which lets us run SQL queries on R tibbles/data frames. Also, to make things go faster, we'll operate on a subsetted version of flights which takes 1% of randomly sampled rows.

### Selecting data from a table
The SQL syntax for selecting column(s) from a table is
```{sql}
SELECT <col1>, <col2>, ..., <coln> FROM <table>
```
Note the similarity to the corresponding `tidyverse` command:
```{r}
select(<table>, <col1>, <col2>, ..., <coln>)
```

The special keyword `*` means "select everything" and is equivalent to `dplyr`'s `everything()`:

If you have a really big table, SQL allows you to `LIMIT` the number of rows it returns.

### Filtering

The SQL syntax for filtering rows in a table uses the `WHERE` clause:
```{sql}
SELECT * FROM <table> WHERE dest = "IAH"
```
This is the same as:
```{r}
filter(<table>, dest == "IAH")
```
Note that SQL uses a single `=` to check equality!

### Missing data
In SQL, missing data is coded as `NULL`. This is a special value which is analogous to `NA` in R. 

### Summarizing

The SQL syntax for summarizing is using the `GROUP BY` clause:
```{sql}
SELECT AVG(<col>) AS avg_col FROM <table> GROUP BY(<group cols>)
```
This is the same as:
```{r}
<table> %>% group_by(<group cols>) %>% summarize(avg_col = mean(<col>))
```

### Joins

The SQL syntax for joins:
```{sql}
SELECT * FROM <table> LEFT JOIN <other_table> ON <left_key_col> = <right_key_col>
```
This is the same as:
```{r}
left_join(<table>, <other_table>, by = c("<left_key_col>" = "<right_key_col>"))
```

Note here that SQL requires us to be explicit about which columns we are `SELECT`ing when joining multiple tables. Each column name must be prefixed with the name of the table in which it resides.

## Advanced joins in SQL
SQL is more general in specifying the join condition. Whereas in tidyverse it must be a key, in
SQL it can be a general logical condition.

## Example
For every airport in `airports`, what is its nearest neighbor?

What is the nearest neighbor to `DTW`?

## Exercise
In 1985 a hurricane made landfall on Long Island, NY between JFK Airport and Islip.
What was the name of that hurricane?